# Prose corpus segmentation

In [2]:
from nltk import word_tokenize
import numpy as np
import pandas as pd
from typing import Dict, List, Optional, Tuple, Union
from utils import build_chunks, split_texts_into_segments

In [9]:
corpus = pd.read_csv("../data/corpora/prose_corpus.csv")

### hyperparameter

Segments: 10000, 25000

In [11]:
segments = 10000

### Splitting into segments

In [18]:
%%time
new_corpus = split_texts_into_segments(corpus, 
                                       n=segments,
                                       same_len=True)
new_corpus.head()

CPU times: user 7min 9s, sys: 2.29 s, total: 7min 11s
Wall time: 7min 14s


,filename,author,title,year,textlength,text
0,Jakob_Christoph_Heer_-_Der_Wetterwart_(1905)_1,Jakob Christoph Heer,Der Wetterwart_1,1905,10000,1925 I Die feierliche Abendhelle steht über de...
1,Jakob_Christoph_Heer_-_Der_Wetterwart_(1905)_2,Jakob Christoph Heer,Der Wetterwart_2,1905,10000,"spüre es , die Ingenieurkunst wäre mein innigs..."
2,Jakob_Christoph_Heer_-_Der_Wetterwart_(1905)_3,Jakob Christoph Heer,Der Wetterwart_3,1905,10000,uns stets ein treuer Nachbar gewesen . « Ich h...
3,Jakob_Christoph_Heer_-_Der_Wetterwart_(1905)_4,Jakob Christoph Heer,Der Wetterwart_4,1905,10000,mit Feierlichkeit . Wozu rief er mich nur ? Al...
4,Jakob_Christoph_Heer_-_Der_Wetterwart_(1905)_5,Jakob Christoph Heer,Der Wetterwart_5,1905,10000,sein Schwager und seine Schwester eine kleine ...


In [19]:
new_corpus.shape

(4736, 6)

In [20]:
#new_corpus.to_csv(f"../data/corpora/prose_corpus_{segments}seg.csv", index=False)